In [1]:
%pip install gradio torch dice

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Genom10\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from src.dnd.units import Unit, GenericSoldier
from src.dnd.game_board import DnDBoard, MovementError
from src.gui.gridDrawer import generate_grid, draw_field, highlight_selected_token
from src.gui.adapters import RenderUnit
from src.dnd.load_unit import load_unit, load_renderUnit, getTokenImagePath, getTokenName

from typing import List
import gradio as gr
import itertools
import glob
import re
import numpy as np

C:\Users\Genom10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
board_size = None
board = None
render_units: List[RenderUnit] = []
selectedToken = None

def getUIDsFromBoard(board: DnDBoard):
  return board.get_UIDs()

def update_unit_list() -> None: 
  global board
  global render_units
  UID2unit = dict(zip([unit.get_UID() for unit in board.units], board.units))

def generate_board(gridScale, gridSize):
  global board_size
  global board
  board_size = gridSize
  board = DnDBoard((board_size, board_size))
  field = generate_grid(board_size, gridScale)
  return field

def add_token(image, x:int, y:int, label, gridScale, jsonDescriptor = None):
  global board_size
  if jsonDescriptor is None:
    board.place_unit(GenericSoldier(), (x,y), 0)
    render_units.append(RenderUnit(board.board[(x,y)].get_UID(), (x,y), image))
  else:
    board.place_unit(load_unit(jsonDescriptor), (x,y), 0)
    r_unit = load_renderUnit(jsonDescriptor, (x,y))
    r_unit.unitUID = board.board[(x,y)].get_UID()
    render_units.append(r_unit)
  if x < board_size:
    x += 1
  else:
    x = 0
    y += 1
  update_unit_list()
  return draw_field(render_units, selectedToken, gridScale, board_size), gr.Number(value=x), gr.Number(value=y), gr.Label(label)

def move_token(index, x:int, y:int, gridScale):
  global board_size
  unit:Unit = board.get_unit_by_UID(render_units[index].getUID())
  try:
    board.move_unit(unit=unit, new_position=(x,y))
    render_units[index].pos = np.array([x, y], dtype = int)
  except MovementError as e:
    print("Error moving into chosen cell:", e)
  return draw_field(render_units, selectedToken, gridScale, board_size)

def set_selected_token(tokenID, gridScale):
  global board_size
  global selectedToken
  selectedToken = tokenID
  return draw_field(render_units, selectedToken, gridScale, board_size)

def on_grid_scale_shange(gridScale):
  global board_size
  return draw_field(render_units, selectedToken, gridScale, board_size)

def update_UID_list():
  new_choices = list(map(lambda x : x.getUID(), render_units))
  new_choices = list(zip(new_choices, itertools.count()))
  return gr.Dropdown(new_choices, interactive=True, label='tokenID')

def on_image_load(currentLabel):
  #TODO read image label
  if currentLabel == '':
    return 
  else:
    return currentLabel 



Interface

In [4]:
with gr.Blocks() as demo:
  #Grid
  with gr.Row():
    gridSize = gr.Slider(label="gridSize", value=10, minimum=2, maximum=100, step=1)
    makeBoard = gr.Button(value="Make Board")
  gridScale = gr.Slider(label="gridScale", value=30, minimum=10, maximum=150, step=1)
  im_canvas = gr.Image()
  makeBoard.click(generate_board, inputs = [gridScale, gridSize], outputs=[im_canvas])
  gridScale.change(on_grid_scale_shange, inputs = [gridScale], outputs=[im_canvas])

  with gr.Tabs(visible=True) as tabs:
    #Add Token
    with gr.TabItem("Add Token") as tab1:
      with gr.Row():
        with gr.Column(min_width=10):
          x = gr.Number(label="x pos", precision=0)
          y = gr.Number(label="y pos", precision=0)
          label = gr.Text()
          jsonDescriptor = gr.File()
        im_in = gr.Image(image_mode='RGBA', type='pil')
        examples_paths = glob.glob('./Tokens/*.json')
        token_data = [[getTokenImagePath(path), getTokenName(path), path] for path in examples_paths]
        print(token_data)
        examples = gr.Examples(examples=token_data, inputs=[im_in, label, jsonDescriptor], examples_per_page=4)
      btn_add = gr.Button(value="Add token")
    im_in.change(on_image_load, inputs=label, outputs=label)
    btn_add.click(add_token, inputs=[im_in, x, y, label, gridScale, jsonDescriptor], outputs=[im_canvas, x, y, label])

    #Move Token
    with gr.TabItem("Move Token") as tab2:
      with gr.Row():
          tokenID = gr.Dropdown(['t1', 't2'], label='tokenID')
          tab2.select(update_UID_list, outputs=tokenID)
          x_move = gr.Number(label="x pos", precision=0)
          y_move = gr.Number(label="y pos", precision=0)
      btn_move = gr.Button(value="Move token")
    tokenID.change(set_selected_token, inputs=[tokenID, gridScale], outputs=[im_canvas])
    btn_move.click(move_token, inputs=[tokenID, x_move, y_move, gridScale], outputs=[im_canvas])


if __name__ == "__main__":
    demo.launch()

[['Tokens\\Squire.png', 'Squire', './Tokens\\Squire.json'], ['Tokens\\Zombie.png', 'Zombie', './Tokens\\Zombie.json']]
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Genom10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Genom10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Genom10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\blocks.py", line 1522, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Genom10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra